In [1]:
# Phase 1: Dataset Curation and Baseline Implementation with AI4M Dataset
# Using real Lean 4 formal-informal pairs and Qwen models

# Cell 1: Install required packages
!pip install -q torch transformers datasets peft bitsandbytes accelerate
!pip install -q sentence-transformers faiss-gpu evaluate rouge-score
!pip install -q sentencepiece protobuf tqdm networkx

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pathos 0.3.4 requires dill>=0.4.0, but you have dill 0.3.8 which is incompatible.
pathos 0.3.4 requires multiprocess>=0.70.18, but you have multiprocess 0.70.16 which is incompatible.
  DEPRECATION: Building 'rouge-score' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'rouge-score'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [1]:

# Cell 2: Setup environment and check GPUs
import os
import torch
import gc
import json
import warnings
from pathlib import Path
warnings.filterwarnings('ignore')

# Set environment variables for efficient GPU usage
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

print("="*60)
print("PHASE 1: MANUAL DATASET & QWEN BASELINES")
print("="*60)

# Check available GPUs
if torch.cuda.is_available():
    print(f"\n✓ CUDA Available")
    for i in range(torch.cuda.device_count()):
        gpu_name = torch.cuda.get_device_name(i)
        gpu_memory = torch.cuda.get_device_properties(i).total_memory / 1e9
        print(f"  GPU {i}: {gpu_name} ({gpu_memory:.2f} GB)")
        
        # Adjust batch size based on GPU memory
        if gpu_memory < 16:  # T4 GPU
            print("    → Detected T4 GPU, using optimized settings")
            os.environ['BATCH_SIZE'] = '1'
        elif gpu_memory < 25:  # L4 GPU
            print("    → Detected L4 GPU, using standard settings")
            os.environ['BATCH_SIZE'] = '2'
else:
    print("⚠ No GPU available, will use CPU (very slow)")

PHASE 1: MANUAL DATASET & QWEN BASELINES

✓ CUDA Available
  GPU 0: NVIDIA L4 (23.57 GB)
    → Detected L4 GPU, using standard settings


In [2]:
# Cell 3: Check and Analyze Dataset Files
print("\n" + "="*60)
print("STEP 1: DATASET ANALYSIS")
print("="*60)

# Define data directory
data_dir = Path("./math_alignment_dataset")
data_dir.mkdir(parents=True, exist_ok=True)

# Check for your manually extracted files
print("\nChecking for dataset files...")

# List all potential data files
data_files = {
    'train': ['statements_part1.json', 'train.json', 'train.jsonl'],
    'valid': ['valid_clean.json', 'val.json', 'val.jsonl'],
    'test': ['test_clean.json', 'test.json', 'test.jsonl']
}

found_files = {}
for split, filenames in data_files.items():
    for filename in filenames:
        filepath = data_dir / filename
        if filepath.exists():
            found_files[split] = filepath
            print(f"✓ Found {split} file: {filename}")
            break
    if split not in found_files:
        print(f"✗ No {split} file found")

# Analyze dataset structure
if 'train' in found_files:
    print("\nAnalyzing training data structure...")
    with open(found_files['train'], 'r', encoding='utf-8') as f:
        content = f.read()
        if content.strip().startswith('['):
            # JSON array format
            train_data = json.loads(content)
        else:
            # JSONL format
            f.seek(0)
            train_data = [json.loads(line) for line in f if line.strip()]
    
    print(f"  Total training samples: {len(train_data)}")
    if train_data:
        print(f"  Sample keys: {list(train_data[0].keys())}")
        
        # Display sample
        sample = train_data[0]
        informal_key = None
        formal_key = None
        
        # Detect keys
        for key in ['informal_statement', 'informal_stmt', 'informal']:
            if key in sample:
                informal_key = key
                break
        
        for key in ['formal_statement', 'formal_stmt', 'formal']:
            if key in sample:
                formal_key = key
                break
        
        if informal_key and formal_key:
            print(f"\n  Sample pair:")
            print(f"  Informal ({informal_key}): {sample[informal_key][:100]}...")
            print(f"  Formal ({formal_key}): {sample[formal_key][:100]}...")



STEP 1: DATASET ANALYSIS

Checking for dataset files...
✓ Found train file: statements_part1.json
✓ Found valid file: valid_clean.json
✓ Found test file: test_clean.json

Analyzing training data structure...
  Total training samples: 702
  Sample keys: ['informal_statement', 'formal_statement']

  Sample pair:
  Informal (informal_statement): For all odd $n$ show that $8 \\mid n^{2}-1$....
  Formal (formal_statement): theorem exercise_1_27 {n : \u2115} (hn : odd n) : 8 \u2223 (n^2 - 1)...


In [3]:
# Cell 4: Configure baselines with Qwen models
print("\n" + "="*60)
print("CONFIGURING QWEN BASELINES")
print("="*60)

from baseline_implementation_manual import (
    BaselineConfig,
    DirectFineTuningBaseline,
    RetrievalAugmentedBaseline,
    CommercialAPIBaseline,
    BaselineEvaluator
)

# Configure for available hardware
config = BaselineConfig()
config.data_dir = str(data_dir)

# Adjust based on GPU memory
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    if gpu_memory < 16:  # T4 GPU
        print("Configuring for T4 GPU (15GB)...")
        config.batch_size = 1
        config.gradient_accumulation = 32
        config.max_length = 256
        config.model_name = "Qwen/Qwen2.5-Math-1.5B"  # Smaller model for T4
        config.api_model = "Qwen/Qwen2.5-0.5B"  # Tiny model for API
        
    elif gpu_memory < 25:  # L4 GPU
        print("Configuring for L4 GPU (24GB)...")
        config.batch_size = 4
        config.gradient_accumulation = 16
        config.max_length = 384
        config.model_name = "Qwen/Qwen2.5-Math-7B"
        config.api_model = "Qwen/Qwen2.5-1.5B"
    else:
        print("Using full configuration...")
        config.batch_size = 4
        config.gradient_accumulation = 8
        config.model_name = "Qwen/Qwen2.5-Math-7B"
        config.api_model = "Qwen/Qwen2.5-3B"

print(f"\nBaseline Configuration:")
print(f"  Fine-tuning model: {config.model_name}")
print(f"  API model: {config.api_model}")
print(f"  Batch size: {config.batch_size}")
print(f"  Max length: {config.max_length}")
print(f"  Data directory: {config.data_dir}")



CONFIGURING QWEN BASELINES
Configuring for L4 GPU (24GB)...

Baseline Configuration:
  Fine-tuning model: Qwen/Qwen2.5-Math-7B
  API model: Qwen/Qwen2.5-1.5B
  Batch size: 4
  Max length: 384
  Data directory: math_alignment_dataset


In [ ]:
# Cell 5: Train Direct Fine-tuning Baseline
print("\n" + "="*60)
print("BASELINE 1: DIRECT FINE-TUNING")
print("="*60)

try:
    print("Initializing Qwen model with QLoRA...")
    direct_baseline = DirectFineTuningBaseline(config)
    
    print("\nStarting training...")
    print("This may take 20-30 minutes on T4, 10-15 minutes on L4...")
    direct_baseline.train()
    
    print("✓ Direct fine-tuning complete!")
    
    # Test the model
    test_inputs = [
        "For all odd n show that 8 divides n^2 - 1",
        "Prove that the square of any real number is non-negative",
        "Show that if a group G has order p^n where p is prime, then G has a subgroup of order p"
    ]
    
    print("\nTest translations:")
    for test_input in test_inputs[:1]:  # Test with first input
        print(f"\nInput: {test_input[:80]}...")
        output = direct_baseline.generate(
            f"Translate to formal: {test_input}\nFormal:",
            max_new_tokens=256
        )
        if "Formal:" in output:
            output = output.split("Formal:")[-1].strip()
        print(f"Output: {output[:150]}...")
    
except Exception as e:
    print(f"❌ Direct fine-tuning failed: {e}")
    import traceback
    traceback.print_exc()

# Clear GPU memory
if 'direct_baseline' in locals():
    del direct_baseline
gc.collect()
torch.cuda.empty_cache()


BASELINE 1: DIRECT FINE-TUNING
Initializing Qwen model with QLoRA...


INFO:baseline_implementation_manual:Loading Qwen/Qwen2.5-Math-7B...
INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:baseline_implementation_manual:Starting training with Qwen2.5-Math...
INFO:baseline_implementation_manual:Found training file: math_alignment_dataset/statements_part1.json
INFO:baseline_implementation_manual:Found validation file: math_alignment_dataset/valid_clean.json
INFO:baseline_implementation_manual:Loaded 702 pairs from math_alignment_dataset/statements_part1.json
INFO:baseline_implementation_manual:Sample item keys: dict_keys(['informal', 'formal'])
INFO:baseline_implementation_manual:Loaded 244 pairs from math_alignment_dataset/valid_clean.json
INFO:baseline_implementation_manual:Sample item keys: dict_keys(['informal', 'formal'])


trainable params: 40,370,176 || all params: 7,655,986,688 || trainable%: 0.5273

Starting training...
This may take 20-30 minutes on T4, 10-15 minutes on L4...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss


In [5]:
# Cell 6: Build Retrieval-Augmented Baseline
print("\n" + "="*60)
print("BASELINE 2: RETRIEVAL-AUGMENTED")
print("="*60)

try:
    print("Building retrieval system...")
    retrieval_baseline = RetrievalAugmentedBaseline(config)
    retrieval_baseline.build_index()
    
    print("✓ Retrieval index built!")
    
    # Test retrieval
    test_formal = "theorem exercise_1_27 {n : ℕ} (hn : odd n) : 8 ∣ (n^2 - 1)"
    result = retrieval_baseline.translate(test_formal, "formal_to_informal")
    print(f"\nTest retrieval:")
    print(f"Input: {test_formal}")
    print(f"Retrieved: {result[:150]}...")
    
except Exception as e:
    print(f"❌ Retrieval baseline failed: {e}")
    import traceback
    traceback.print_exc()

if 'retrieval_baseline' in locals():
    del retrieval_baseline
gc.collect()
torch.cuda.empty_cache()

INFO:baseline_implementation_manual:Loading embedding model: sentence-transformers/all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda:0
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2



BASELINE 2: RETRIEVAL-AUGMENTED
Building retrieval system...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:baseline_implementation_manual:Building FAISS indices...
INFO:baseline_implementation_manual:Indices saved to baseline_output/retrieval_augmented


✓ Retrieval index built!


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Test retrieval:
Input: theorem exercise_1_27 {n : ℕ} (hn : odd n) : 8 ∣ (n^2 - 1)
Retrieved: For all odd $n$ show that $8 \\mid n^{2}-1$....


In [6]:
# Cell 7: Setup API Baseline
print("\n" + "="*60)
print("BASELINE 3: API BASELINE (QWEN)")
print("="*60)

try:
    print(f"Loading {config.api_model}...")
    api_baseline = CommercialAPIBaseline(config)
    
    print("✓ API baseline ready!")
    
    # Test API baseline
    test_informal = "The square of any real number is non-negative"
    result = api_baseline.translate_with_cot(test_informal, "informal_to_formal")
    print(f"\nTest API translation:")
    print(f"Input: {test_informal}")
    print(f"Output: {result[:150]}...")
    
except Exception as e:
    print(f"❌ API baseline setup failed: {e}")
    import traceback
    traceback.print_exc()

if 'api_baseline' in locals():
    del api_baseline
gc.collect()
torch.cuda.empty_cache()


INFO:baseline_implementation_manual:Loading API model: Qwen/Qwen2.5-1.5B



BASELINE 3: API BASELINE (QWEN)
Loading Qwen/Qwen2.5-1.5B...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

✓ API baseline ready!

Test API translation:
Input: The square of any real number is non-negative
Output: For any real number x, the square of x (denoted as x^2) is greater than or equal to 0.

Justification:

1. I identified that the informal statement is...


In [8]:
!pip install bert_score -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# Cell 8: Evaluate All Baselines
print("\n" + "="*60)
print("EVALUATION PHASE")
print("="*60)

try:
    print("Starting comprehensive evaluation...")
    print("This will evaluate all baselines on the test set")
    
    evaluator = BaselineEvaluator(config)
    all_results = {}
    
    # Check if test file exists
    test_file = None
    test_files = [
        f"{config.data_dir}/test_clean.json",
        f"{config.data_dir}/test.json",
        f"{config.data_dir}/test.jsonl"
    ]
    
    for f in test_files:
        if Path(f).exists():
            test_file = f
            print(f"Found test file: {f}")
            break
    
    if not test_file:
        print("⚠ No test file found, using validation set for evaluation")
        test_file = f"{config.data_dir}/valid_clean.json"
    
    # Evaluate Direct Fine-tuning
    print("\n1. Evaluating Direct Fine-tuning...")
    try:
        direct_baseline = DirectFineTuningBaseline(config)
        # Load trained model if exists
        model_path = Path(config.output_dir) / "direct_finetuning" / "final_model"
        if model_path.exists():
            from peft import PeftModel
            direct_baseline.model = PeftModel.from_pretrained(
                direct_baseline.model, str(model_path)
            )
            print("  Loaded trained model")
        
        direct_results = evaluator.evaluate_model(
            direct_baseline,
            test_file,
            "direct_finetuning"
        )
        all_results["direct_finetuning"] = direct_results
        del direct_baseline
        gc.collect()
        torch.cuda.empty_cache()
    except Exception as e:
        print(f"  Failed: {e}")
    
    # Evaluate Retrieval
    print("\n2. Evaluating Retrieval-Augmented...")
    try:
        retrieval_baseline = RetrievalAugmentedBaseline(config)
        index_path = Path(config.output_dir) / "retrieval_augmented" / "formal_index.faiss"
        if index_path.exists():
            import faiss
            retrieval_baseline.formal_index = faiss.read_index(str(index_path))
            retrieval_baseline.informal_index = faiss.read_index(
                str(Path(config.output_dir) / "retrieval_augmented" / "informal_index.faiss")
            )
            print("  Loaded existing indices")
            
            # Reload texts
            train_file = found_files.get('train')
            if train_file:
                with open(train_file, 'r', encoding='utf-8') as f:
                    content = f.read()
                    if content.strip().startswith('['):
                        data = json.loads(content)
                    else:
                        f.seek(0)
                        data = [json.loads(line) for line in f if line.strip()]
                
                for item in data:
                    for inf_key in ['informal_statement', 'informal_stmt', 'informal']:
                        if inf_key in item:
                            for form_key in ['formal_statement', 'formal_stmt', 'formal']:
                                if form_key in item:
                                    retrieval_baseline.formal_texts.append(item[form_key])
                                    retrieval_baseline.informal_texts.append(item[inf_key])
                                    break
                            break
        else:
            retrieval_baseline.build_index()
        
        retrieval_results = evaluator.evaluate_model(
            retrieval_baseline,
            test_file,
            "retrieval_augmented"
        )
        all_results["retrieval_augmented"] = retrieval_results
        del retrieval_baseline
        gc.collect()
    except Exception as e:
        print(f"  Failed: {e}")
    
    # Evaluate API
    print("\n3. Evaluating API Baseline...")
    try:
        api_baseline = CommercialAPIBaseline(config)
        api_results = evaluator.evaluate_model(
            api_baseline,
            test_file,
            "api_baseline"
        )
        all_results["api_baseline"] = api_results
        del api_baseline
        gc.collect()
        torch.cuda.empty_cache()
    except Exception as e:
        print(f"  Failed: {e}")
    
    print("\n✓ Evaluation complete!")
    
    # Save combined results
    if all_results:
        results_dir = Path(config.output_dir) / "evaluation_results"
        results_dir.mkdir(parents=True, exist_ok=True)
        with open(results_dir / "all_results.json", 'w') as f:
            json.dump(all_results, f, indent=2)
    
except Exception as e:
    print(f"❌ Evaluation failed: {e}")
    import traceback
    traceback.print_exc()


EVALUATION PHASE
Starting comprehensive evaluation...
This will evaluate all baselines on the test set


INFO:matplotlib.font_manager:generated new fontManager


Found test file: math_alignment_dataset/test_clean.json

1. Evaluating Direct Fine-tuning...


INFO:baseline_implementation_manual:Loading Qwen/Qwen2.5-Math-7B...
INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating retrieval_augmented: 100%|██████████| 50/50 [00:01<00:00, 42.88it/s]
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:baseline_implementation_manual:Results saved to baseline_output/evaluation_results/retrieval_augmented_results.json
INFO:baseline_implementation_manual:Loading API model: Qwen/Qwen2.5-1.5B



3. Evaluating API Baseline...


INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
INFO:baseline_implementation_manual:Evaluating api_baseline...
INFO:baseline_implementation_manual:Found test file: math_alignment_dataset/test_clean.json
Evaluating api_baseline: 100%|██████████| 50/50 [13:06<00:00, 15.74s/it]
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:baseline_implementation_manual:Results saved to baseline_output/evaluation_results/api_baseline_results.json



✓ Evaluation complete!


In [10]:
# Cell 9: Display Results Summary
print("\n" + "="*60)
print("RESULTS SUMMARY")
print("="*60)

results_dir = Path(config.output_dir) / "evaluation_results"

if results_dir.exists():
    # Load all results
    all_results_file = results_dir / "all_results.json"
    if all_results_file.exists():
        with open(all_results_file, 'r') as f:
            all_results = json.load(f)
        
        # Display results
        print("\nBaseline Performance on Manual Dataset:")
        print("-" * 60)
        
        for model_name, model_results in all_results.items():
            print(f"\n{model_name.replace('_', ' ').title()}:")
            for direction in ['formal_to_informal', 'informal_to_formal']:
                if direction in model_results:
                    metrics = model_results[direction]
                    dir_label = 'F→I' if direction == 'formal_to_informal' else 'I→F'
                    print(f"  {dir_label}:")
                    print(f"    BLEU:      {metrics.get('bleu', 0):.4f}")
                    print(f"    ROUGE-L:   {metrics.get('rougeL', 0):.4f}")
                    print(f"    BERTScore: {metrics.get('bertscore_f1', 0):.4f}")
    else:
        print("No results file found")

# Cell 10: Analysis and Next Steps
print("\n" + "="*60)
print("PHASE 1 COMPLETE!")
print("="*60)

print("\n📊 Key Achievements:")
print("1. ✓ Loaded your manually extracted Lean 4 formal-informal pairs")
print("2. ✓ Trained Qwen2.5-Math model with QLoRA for efficient fine-tuning")
print("3. ✓ Built retrieval system with real mathematical theorems")
print("4. ✓ Established baselines with proper evaluation metrics")

print("\n🎯 Dataset Summary:")
if 'train' in found_files:
    print(f"• Training samples: {len(train_data)} pairs")
print(f"• Column format detected: {informal_key} / {formal_key}")

print("\n🚀 Next Steps for Phase 2 (FIRMA):")
print("1. Use this dataset for FIRMA training")
print("2. Implement bidirectional translation with complexity control")
print("3. Compare FIRMA against these Qwen baselines")
print("4. Achieve superior performance through architectural innovations")

print("\n💡 Tips:")
print("• If results are low, check that train/valid/test files use consistent column names")
print("• Consider increasing training epochs if GPU memory allows")
print("• For better retrieval, use a math-specific embedding model")

# Cell 11: Save dataset statistics
print("\n" + "="*60)
print("SAVING DATASET STATISTICS")
print("="*60)

stats = {
    "found_files": {k: str(v) for k, v in found_files.items()},
    "train_samples": len(train_data) if 'train_data' in locals() else 0,
    "column_format": {
        "informal": informal_key if 'informal_key' in locals() else None,
        "formal": formal_key if 'formal_key' in locals() else None
    },
    "gpu_config": {
        "available": torch.cuda.is_available(),
        "device_count": torch.cuda.device_count() if torch.cuda.is_available() else 0,
        "memory_gb": torch.cuda.get_device_properties(0).total_memory / 1e9 if torch.cuda.is_available() else 0
    }
}

stats_file = Path(config.output_dir) / "dataset_stats.json"
with open(stats_file, 'w') as f:
    json.dump(stats, f, indent=2)

print(f"✓ Statistics saved to {stats_file}")
print("\nPhase 1 notebook execution complete!")


RESULTS SUMMARY

Baseline Performance on Manual Dataset:
------------------------------------------------------------

Direct Finetuning:
  F→I:
    BLEU:      0.0771
    ROUGE-L:   0.2090
    BERTScore: 0.8244
  I→F:
    BLEU:      0.0309
    ROUGE-L:   0.1651
    BERTScore: 0.7873

Retrieval Augmented:
  F→I:
    BLEU:      0.0238
    ROUGE-L:   0.1422
    BERTScore: 0.8231
  I→F:
    BLEU:      0.0154
    ROUGE-L:   0.1554
    BERTScore: 0.8130

Api Baseline:
  F→I:
    BLEU:      0.0483
    ROUGE-L:   0.2235
    BERTScore: 0.8053
  I→F:
    BLEU:      0.0320
    ROUGE-L:   0.1401
    BERTScore: 0.7875

PHASE 1 COMPLETE!

📊 Key Achievements:
1. ✓ Loaded your manually extracted Lean 4 formal-informal pairs
2. ✓ Trained Qwen2.5-Math model with QLoRA for efficient fine-tuning
3. ✓ Built retrieval system with real mathematical theorems
4. ✓ Established baselines with proper evaluation metrics

🎯 Dataset Summary:
• Training samples: 702 pairs
• Column format detected: informal_statement